In [ ]:
# =============================================================================
# Домашняя работа: Кластеризация и тематическое моделирование коротких текстов
# Выполнил: Головастиков Юрий Юрьевич
# Дата: 12 января 2026
# =============================================================================

# Установка зависимостей (запустить один раз)
# !pip install -q bertopic datasets sentence-transformers hdbscan umap-learn datamapplot wordcloud pyLDAvis pymorphy3 nltk plotly

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm.auto import tqdm

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import datamapplot

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
import re

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

ru_stop = set(stopwords.words('english') + stopwords.words('russian'))
morph = pymorphy3.MorphAnalyzer()

%matplotlib inline
plt.style.use('seaborn-v0_8-pastel')

print("Библиотеки загружены\n")

# ─── 1. Загрузка и предобработка данных ─────────────────────────────────────
print("1. Загрузка датасета коротких текстов (BBC News)...")
dataset = load_dataset("SetFit/bbc-news", split="train")
texts = [row['text'] for row in dataset if row['text'] and len(row['text']) > 100]
texts = texts[:3500]  # берём 3500 для скорости и качества

print(f"Всего текстов: {len(texts):,}\n")

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)  # только буквы и пробелы
    tokens = word_tokenize(text)
    tokens = [morph.parse(t)[0].normal_form for t in tokens 
              if t not in ru_stop and len(t) > 2]
    return tokens

print("Предобработка текстов...")
processed = [preprocess(t) for t in tqdm(texts)]

# Добавляем биграммы
bigram = Phrases(processed, min_count=3, threshold=8)
bigram_mod = Phraser(bigram)
processed = [bigram_mod[doc] for doc in processed]

print("Пример обработанного текста:")
print(" ".join(processed[0][:20]) + "...\n")

# ─── 2. Эмбеддинги (мультиязычная модель) ───────────────────────────────────
print("2. Получаем эмбеддинги...")
embed_model = SentenceTransformer("intfloat/multilingual-e5-large")
embeddings = embed_model.encode(texts, show_progress_bar=True, batch_size=64)

print(f"Размерность эмбеддингов: {embeddings.shape}\n")

# ─── 3. Кластеризация HDBSCAN ───────────────────────────────────────────────
print("3. Кластеризация с HDBSCAN...")
hdbscan_model = HDBSCAN(
    min_cluster_size=45,
    min_samples=12,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

clusters = hdbscan_model.fit_predict(embeddings)

n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
noise_pct = list(clusters).count(-1) / len(clusters) * 100

print(f"Выделено кластеров: {n_clusters}")
print(f"Шумовых точек: {noise_pct:.1f}%")

# Размеры кластеров
cluster_sizes = pd.Series(clusters).value_counts().sort_index()
print("\nРазмеры кластеров:")
print(cluster_sizes.head(15))

# Примеры текстов из топ-кластера
top_cluster = cluster_sizes.index[1]  # самый большой ненулевой
print(f"\nПримеры из самого большого кластера ({top_cluster}):")
for idx in np.where(clusters == top_cluster)[0][:3]:
    print(f"  • {texts[idx][:180]}...\n")